### Imports

In [1]:
import pickle
import tensorflow as tf
import random
from itertools import combinations
import numpy as np
from tensorflow.keras.layers import GRU, ConvLSTM2D, Input, Flatten, Dense, Dropout, Lambda, Conv2D, MaxPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras import Model
import tensorflow.keras.backend as K
import librosa
from tensorflow.keras.utils import Sequence
from tensorflow.keras.activations import sigmoid
from tensorflow.keras.metrics import BinaryAccuracy

/home/rushabh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rushabh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rushabh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rushabh/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

### Load Training and Testing Data

In [2]:
with open('hw4_trs.pkl', 'rb') as pickle_file:
    train_data = pickle.load(pickle_file)
print(train_data.shape)

(500, 16180)


In [3]:
with open('hw4_tes.pkl', 'rb') as pickle_file:
    test_data = pickle.load(pickle_file)
print(test_data.shape)

(200, 22631)


### Create Positive Pairs
Generates all combinations of pairs and randomly selects L pairs from it.

In [4]:
def create_pos_pairs(speaker, L=45):
    batch = []
    all_pairs = list(combinations(range(10), 2))
    l_pairs = random.sample(all_pairs, L)
    for a,b in l_pairs:
        stft_1 = np.abs(librosa.stft(speaker[a], n_fft=1024, hop_length=512)).T
        stft_2 = np.abs(librosa.stft(speaker[b], n_fft=1024, hop_length=512)).T
        batch.append([stft_1, stft_2])
    return batch

### Create Negative Pairs
The Main Speaker's utternaces lie between start and end. The other 49 speakers' utterances like before start and after end.  
Randomly sample L utterances from the main speaker's utterances with replacement as L is greater than the number of utterances by the main speaker, i.e., L>10.  
Whereas, we randomly sample L utterances from the other 49 speakers' utterances without replacement as L is less than their total utterances L<490.
Using the samples from the above 2 steps we create Negative Pairs

In [5]:
def create_neg_pairs(pos_sp_num, data, L=45):
    batch = []
    
    start = pos_sp_num*10
    end = start + 10     
    pos_spk = data[start: end]
    neg_spk = data[:start] + data[end:]
    neg_sample = random.sample(neg_spk, L)
        
    for l in range(L):
        pos = random.choice(pos_spk)
        
        stft_pos = np.abs(librosa.stft(pos, n_fft=1024, hop_length=512)).T
        stft_neg = np.abs(librosa.stft(neg_sample[l], n_fft=1024, hop_length=512)).T
        batch.append([stft_pos, stft_neg])

    return batch

### Create Training Pairs
Creates 2250 pairs of negative and positive examples each, i.e, 4500 total pairs.
Final Dimensions - Pairs (4500) x Number of Inputs (2) x Shape of Spectrogram (45x513) x number of channels (1)

In [6]:
mini_batches = []
train_data = list(train_data)
for i in range(50):
    pos_batch = create_pos_pairs(train_data[i:i+10])
    neg_batch = create_neg_pairs(i, train_data)
    mini_batches += pos_batch + neg_batch
#     mini_batches.append(mini_batch)

In [7]:
mini_np = np.stack(mini_batches)

In [8]:
mini_np.shape

(4500, 2, 32, 513)

In [9]:
mini_np = np.pad(mini_np, ((0,0), (0,0), (0, 13), (0,0)), mode = 'constant', constant_values = 0) 
print(mini_np.shape)

(4500, 2, 45, 513)


In [ ]:
# mini_np_exp = np.expand_dims(mini_np, axis = 4)

In [ ]:
# mini_np_exp.shape

### Create True Predictions based on the negative/positive pairs

In [10]:
Y = [] 
for i in range(50):
    y = np.zeros(90, dtype = int)
    y[:45] += 1
    Y.append(y)
Y = np.hstack(Y)

### Create Testing Pairs
Creates 900 pairs of negative and positive examples each, i.e, 1800 total pairs.  
Final Dimensions - Pairs (1800) x Number of Inputs (2) x Shape of Spectrogram (45x513) x number of channels (1)

In [11]:
mini_batches_test = []
test_data = list(test_data)
for i in range(20):
    pos_batch = create_pos_pairs(test_data[i:i+10])
    neg_batch = create_neg_pairs(i, test_data)
    mini_batches_test += pos_batch + neg_batch

In [12]:
mini_np_test = np.stack(mini_batches_test)

In [13]:
mini_np_test.shape

(1800, 2, 45, 513)

In [ ]:
# mini_np_exp_test = np.expand_dims(mini_np_test, axis = 4)
# mini_np_exp_test.shape

### Loss Function
Binary Cross Entropy aka Sigmoid Cross Entropy is used

In [14]:
def loss(y_true, y_pred):
    return tf.reduce_sum(tf.losses.sigmoid_cross_entropy(y_true, y_pred))

In [15]:
def siamese_model(input_shape):
    left_input = Input(input_shape)
    right_input = Input(input_shape)
    
    # Base Network
    base_model = Sequential()
    base_model.add(GRU(513, return_sequences = False))
    base_model.add(Dropout(rate = 0.2))
    base_model.add(Dense(200, activation = 'tanh'))
    base_model.add(Dropout(rate = 0.2))
    base_model.add(Dense(50, activation = 'tanh'))
    
    left_output = base_model(left_input)
    right_output = base_model(right_input)
    
    print(K.shape(left_output))
    print(K.shape(right_output))
    
    prediction = tf.reduce_sum(tf.multiply(left_output, right_output), axis = 1)
    prediction = tf.reshape(prediction, [-1, 1])
    
    siamese_model = Model(inputs = [left_input, right_input], outputs = prediction)
    
    return siamese_model

In [16]:
model = siamese_model([45, 513])
model.compile(loss=loss, optimizer = tf.keras.optimizers.Adam(), metrics=[BinaryAccuracy()])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Tensor("Shape:0", shape=(2,), dtype=int32)
Tensor("Shape_1:0", shape=(2,), dtype=int32)
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [17]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 45, 513)]    0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 45, 513)]    0                                            
__________________________________________________________________________________________________
sequential (Sequential)         (None, 50)           1693403     input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
tf_op_layer_Mul (TensorFlowOpLa [(None, 50)]         0           sequential[0][0]             

In [18]:
Y_test = [] 
for i in range(20):
    y = np.zeros(90, dtype = int)
    y[:45] += 1
    Y_test.append(y)
Y_test = np.hstack(Y_test)

In [19]:
model.fit([mini_np[:, 0], mini_np[:, 1]], Y, batch_size=32, epochs=500, validation_data=([mini_np_test[:, 0], mini_np_test[:, 1]] ,Y_test))

Train on 4500 samples, validate on 1800 samples
Epoch 1/500
4500/4500 [==============================] - 79s 18ms/sample - loss: 0.6828 - binary_accuracy: 0.5264 - val_loss: 8.2937 - val_binary_accuracy: 0.5000
Epoch 2/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.5094 - binary_accuracy: 0.8593 - val_loss: 3.7110 - val_binary_accuracy: 0.5028
Epoch 3/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.4287 - binary_accuracy: 0.9420 - val_loss: 2.9624 - val_binary_accuracy: 0.5167
Epoch 4/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.3946 - binary_accuracy: 0.9618 - val_loss: 5.0024 - val_binary_accuracy: 0.4939
Epoch 5/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.3851 - binary_accuracy: 0.9636 - val_loss: 3.8864 - val_binary_accuracy: 0.4983
Epoch 6/500
4500/4500 [==============================] - 79s 17ms/sample - loss: 0.3787 - binary_accuracy: 0.9616 - val_loss: 3.8427 - va

Epoch 51/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.3212 - binary_accuracy: 0.9536 - val_loss: 3.3084 - val_binary_accuracy: 0.5217
Epoch 52/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.3014 - binary_accuracy: 0.9567 - val_loss: 4.9479 - val_binary_accuracy: 0.5056
Epoch 53/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.3123 - binary_accuracy: 0.9718 - val_loss: 4.6854 - val_binary_accuracy: 0.5017
Epoch 54/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.3111 - binary_accuracy: 0.9564 - val_loss: 3.9763 - val_binary_accuracy: 0.5117
Epoch 55/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.2942 - binary_accuracy: 0.9578 - val_loss: 3.7626 - val_binary_accuracy: 0.5028
Epoch 56/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.2906 - binary_accuracy: 0.9604 - val_loss: 3.6719 - val_binary_accuracy: 0.5083
Epoch 57/500
450

Epoch 101/500
4500/4500 [==============================] - 83s 18ms/sample - loss: 0.1987 - binary_accuracy: 0.9447 - val_loss: 7.3652 - val_binary_accuracy: 0.5017
Epoch 102/500
4500/4500 [==============================] - 84s 19ms/sample - loss: 0.2427 - binary_accuracy: 0.9444 - val_loss: 6.2544 - val_binary_accuracy: 0.5133
Epoch 103/500
4500/4500 [==============================] - 84s 19ms/sample - loss: 0.2545 - binary_accuracy: 0.9504 - val_loss: 7.6970 - val_binary_accuracy: 0.5050
Epoch 104/500
4500/4500 [==============================] - 84s 19ms/sample - loss: 0.3299 - binary_accuracy: 0.9516 - val_loss: 4.0378 - val_binary_accuracy: 0.4733
Epoch 105/500
4500/4500 [==============================] - 84s 19ms/sample - loss: 0.2712 - binary_accuracy: 0.9484 - val_loss: 4.5074 - val_binary_accuracy: 0.5083
Epoch 106/500
4500/4500 [==============================] - 80s 18ms/sample - loss: 0.2486 - binary_accuracy: 0.9436 - val_loss: 4.4153 - val_binary_accuracy: 0.5156
Epoch 107/

4500/4500 [==============================] - 79s 18ms/sample - loss: 0.2403 - binary_accuracy: 0.9402 - val_loss: 5.8112 - val_binary_accuracy: 0.5111
Epoch 151/500
4500/4500 [==============================] - 79s 18ms/sample - loss: 0.3632 - binary_accuracy: 0.9051 - val_loss: 5.5442 - val_binary_accuracy: 0.5022
Epoch 152/500
4500/4500 [==============================] - 79s 18ms/sample - loss: 0.2736 - binary_accuracy: 0.9407 - val_loss: 4.7134 - val_binary_accuracy: 0.4967
Epoch 153/500
4500/4500 [==============================] - 79s 18ms/sample - loss: 0.2142 - binary_accuracy: 0.9473 - val_loss: 5.7947 - val_binary_accuracy: 0.5078
Epoch 154/500
4500/4500 [==============================] - 79s 18ms/sample - loss: 0.2034 - binary_accuracy: 0.9402 - val_loss: 5.7146 - val_binary_accuracy: 0.5039
Epoch 155/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.1634 - binary_accuracy: 0.9531 - val_loss: 7.2522 - val_binary_accuracy: 0.5061
Epoch 156/500
4500/4500 

Epoch 200/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.1324 - binary_accuracy: 0.9651 - val_loss: 3.8763 - val_binary_accuracy: 0.5039
Epoch 201/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.0900 - binary_accuracy: 0.9822 - val_loss: 3.8385 - val_binary_accuracy: 0.5200
Epoch 202/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.0972 - binary_accuracy: 0.9838 - val_loss: 4.1364 - val_binary_accuracy: 0.5178
Epoch 203/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.0635 - binary_accuracy: 0.9876 - val_loss: 4.2772 - val_binary_accuracy: 0.5033
Epoch 204/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.0793 - binary_accuracy: 0.9816 - val_loss: 5.3811 - val_binary_accuracy: 0.5106
Epoch 205/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.1724 - binary_accuracy: 0.9582 - val_loss: 4.0882 - val_binary_accuracy: 0.5300
Epoch 206/

...

Epoch 299/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.1310 - binary_accuracy: 0.9738 - val_loss: 2.4854 - val_binary_accuracy: 0.5411
Epoch 300/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.1256 - binary_accuracy: 0.9822 - val_loss: 2.3808 - val_binary_accuracy: 0.5294
Epoch 301/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.1223 - binary_accuracy: 0.9756 - val_loss: 2.5093 - val_binary_accuracy: 0.5478
Epoch 302/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.1231 - binary_accuracy: 0.9804 - val_loss: 2.6055 - val_binary_accuracy: 0.5261
Epoch 303/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.0794 - binary_accuracy: 0.9900 - val_loss: 2.6698 - val_binary_accuracy: 0.5372
Epoch 304/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.0565 - binary_accuracy: 0.9918 - val_loss: 2.9158 - val_binary_accuracy: 0.5344
Epoch 305/

4500/4500 [==============================] - 78s 17ms/sample - loss: 0.0731 - binary_accuracy: 0.9867 - val_loss: 2.6360 - val_binary_accuracy: 0.5528
Epoch 349/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.0642 - binary_accuracy: 0.9918 - val_loss: 2.7491 - val_binary_accuracy: 0.5506
Epoch 350/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.1459 - binary_accuracy: 0.9771 - val_loss: 3.5652 - val_binary_accuracy: 0.5244
Epoch 351/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.1827 - binary_accuracy: 0.9622 - val_loss: 3.0868 - val_binary_accuracy: 0.5283
Epoch 352/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.1764 - binary_accuracy: 0.9642 - val_loss: 3.0210 - val_binary_accuracy: 0.5217
Epoch 353/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.1441 - binary_accuracy: 0.9831 - val_loss: 3.6079 - val_binary_accuracy: 0.5217
Epoch 354/500
4500/4500 

Epoch 398/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.1189 - binary_accuracy: 0.9778 - val_loss: 2.8407 - val_binary_accuracy: 0.5267
Epoch 399/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.1190 - binary_accuracy: 0.9767 - val_loss: 2.8722 - val_binary_accuracy: 0.5500
Epoch 400/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.0862 - binary_accuracy: 0.9840 - val_loss: 3.4429 - val_binary_accuracy: 0.5356
Epoch 401/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.0853 - binary_accuracy: 0.9887 - val_loss: 2.9216 - val_binary_accuracy: 0.5189
Epoch 402/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.0455 - binary_accuracy: 0.9922 - val_loss: 3.5576 - val_binary_accuracy: 0.5067
Epoch 403/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.0519 - binary_accuracy: 0.9916 - val_loss: 3.6746 - val_binary_accuracy: 0.5311
Epoch 404/

4500/4500 [==============================] - 78s 17ms/sample - loss: 0.1341 - binary_accuracy: 0.9771 - val_loss: 3.4517 - val_binary_accuracy: 0.5072
Epoch 448/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.0705 - binary_accuracy: 0.9869 - val_loss: 3.8553 - val_binary_accuracy: 0.4972
Epoch 449/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.0335 - binary_accuracy: 0.9947 - val_loss: 3.7115 - val_binary_accuracy: 0.5106
Epoch 450/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.0219 - binary_accuracy: 0.9956 - val_loss: 4.0924 - val_binary_accuracy: 0.5072
Epoch 451/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.0408 - binary_accuracy: 0.9929 - val_loss: 4.2815 - val_binary_accuracy: 0.5039
Epoch 452/500
4500/4500 [==============================] - 78s 17ms/sample - loss: 0.2395 - binary_accuracy: 0.9469 - val_loss: 4.5482 - val_binary_accuracy: 0.5061
Epoch 453/500
4500/4500 

Epoch 497/500
4500/4500 [==============================] - 79s 18ms/sample - loss: 0.3000 - binary_accuracy: 0.9076 - val_loss: 2.7538 - val_binary_accuracy: 0.4972
Epoch 498/500
4500/4500 [==============================] - 80s 18ms/sample - loss: 0.2699 - binary_accuracy: 0.9242 - val_loss: 2.9735 - val_binary_accuracy: 0.5028
Epoch 499/500
4500/4500 [==============================] - 79s 18ms/sample - loss: 0.2248 - binary_accuracy: 0.9422 - val_loss: 3.2015 - val_binary_accuracy: 0.5122
Epoch 500/500
4500/4500 [==============================] - 79s 18ms/sample - loss: 0.2140 - binary_accuracy: 0.9489 - val_loss: 3.2657 - val_binary_accuracy: 0.4933


In [21]:
model.evaluate([mini_np_test[:, 0], mini_np_test[:, 1]] ,Y[:1800])

1800/1800 [==============================] - 6s 3ms/sample - loss: 3.2657 - binary_accuracy: 0.4933


[3.265663581678851, 0.49333334]

Things I tried
Models - 
1. Stacking Convolution Layers
2. Stacking GRUs
3. ConvLSTM2D